In [1]:
# Etapa 1: Query SQL para histórico ponto folha fechada

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/lk_ponto.Lakehouse/Tables/tab_gold_fato_painel_ponto_folha_fechada"

aafhtfun = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_AutomacaoPonto.Lakehouse/Tables/aafhtfun"
)
aafhtfun.createOrReplaceTempView("aafhtfun")

aabonfun = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_AutomacaoPonto.Lakehouse/Tables/aabonfun"
)
aabonfun.createOrReplaceTempView("aabonfun")

tab_gold_dim_funcionario_historico = spark.read.format("delta").load(
  "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/lk_departamento_pessoal.Lakehouse/Tables/tab_gold_dim_funcionario_historico"
)
tab_gold_dim_funcionario_historico.createOrReplaceTempView("tab_gold_dim_funcionario_historico")


df_resultado = spark.sql("""
SELECT 
    fh.*,

    /*------------------------------------------------------------------------*/
    CASE
        WHEN MONTH(aafhtfun.DATA) < 12 THEN YEAR(aafhtfun.DATA)
        WHEN MONTH(aafhtfun.DATA) = 12 AND DAY(aafhtfun.DATA) < 21 THEN YEAR(aafhtfun.DATA)
        WHEN MONTH(aafhtfun.DATA) = 12 AND DAY(aafhtfun.DATA) >= 21 THEN YEAR(aafhtfun.DATA) + 1
    END                                                                     AS COMPETENCIA_ANO,

    /*------------------------------------------------------------------------*/
    CASE
        WHEN DAY(aafhtfun.DATA) < 21 THEN MONTH(aafhtfun.DATA)
        ELSE
            CASE
                WHEN MONTH(aafhtfun.DATA) + 1 = 13 THEN 1
                ELSE MONTH(aafhtfun.DATA) + 1
            END
    END                                                                     AS COMPETENCIA_MES,

    /*------------------------------------------------------------------------*/
    date_format(aafhtfun.DATA, 'dd/MM/yyyy')                                AS BATIDA_DATA,
    YEAR(aafhtfun.DATA)                                                     AS BATIDA_ANO,
    MONTH(aafhtfun.DATA)                                                    AS BATIDA_MES,
    DAY(aafhtfun.DATA)                                                      AS BATIDA_DIA,

    /*------------------------------------------------------------------------*/
    aafhtfun.HTRAB                                                          AS MIN_HORA_BASE,
    (CASE
        WHEN (aafhtfun.BASE / 60) > 9 THEN CAST((aafhtfun.BASE / 60) AS STRING)
        ELSE '0' || CAST((aafhtfun.BASE / 60) AS STRING)
     END) || ':' ||
     (CASE
        WHEN (aafhtfun.BASE - ((aafhtfun.BASE / 60) * 60)) > 9
            THEN CAST((aafhtfun.BASE - ((aafhtfun.BASE / 60) * 60)) AS STRING)
        ELSE '0' || CAST((aafhtfun.BASE - ((aafhtfun.BASE / 60) * 60)) AS STRING)
     END)                                                                   AS HORA_BASE,

    /*------------------------------------------------------------------------*/
    aafhtfun.HTRAB                                                          AS MIN_HORA_TRABALHADA,
    (CASE
        WHEN (aafhtfun.HTRAB / 60) > 9 THEN CAST((aafhtfun.HTRAB / 60) AS STRING)
        ELSE '0' || CAST((aafhtfun.HTRAB / 60) AS STRING)
     END) || ':' ||
     (CASE
        WHEN (aafhtfun.HTRAB - ((aafhtfun.HTRAB / 60) * 60)) > 9
            THEN CAST((aafhtfun.HTRAB - ((aafhtfun.HTRAB / 60) * 60)) AS STRING)
        ELSE '0' || CAST((aafhtfun.HTRAB - ((aafhtfun.HTRAB / 60) * 60)) AS STRING)
     END)                                                                   AS HORAS_TRABALHADAS,

    /*------------------------------------------------------------------------*/
    aafhtfun.EXTRAEXECUTADO                                                 AS MIN_HORA_EXTRA,
    (CASE
        WHEN (aafhtfun.EXTRAEXECUTADO / 60) > 9 THEN CAST((aafhtfun.EXTRAEXECUTADO / 60) AS STRING)
        ELSE '0' || CAST((aafhtfun.EXTRAEXECUTADO / 60) AS STRING)
     END) || ':' ||
     (CASE
        WHEN (aafhtfun.EXTRAEXECUTADO - ((aafhtfun.EXTRAEXECUTADO / 60) * 60)) > 9
            THEN CAST((aafhtfun.EXTRAEXECUTADO - ((aafhtfun.EXTRAEXECUTADO / 60) * 60)) AS STRING)
        ELSE '0' || CAST((aafhtfun.EXTRAEXECUTADO - ((aafhtfun.EXTRAEXECUTADO / 60) * 60)) AS STRING)
     END)                                                                   AS HORAS_EXTRAS,

    /*------------------------------------------------------------------------*/
    aafhtfun.ATRASO                                                         AS MIN_ATRASO,
    (CASE
        WHEN (aafhtfun.ATRASO / 60) > 9 THEN CAST((aafhtfun.ATRASO / 60) AS STRING)
        ELSE '0' || CAST((aafhtfun.ATRASO / 60) AS STRING)
     END) || ':' ||
     (CASE
        WHEN (aafhtfun.ATRASO - ((aafhtfun.ATRASO / 60) * 60)) > 9
            THEN CAST((aafhtfun.ATRASO - ((aafhtfun.ATRASO / 60) * 60)) AS STRING)
        ELSE '0' || CAST((aafhtfun.ATRASO - ((aafhtfun.ATRASO / 60) * 60)) AS STRING)
     END)                                                                   AS ATRASO,

    /*------------------------------------------------------------------------*/
    aafhtfun.FALTA                                                          AS MIN_FALTA,
    (CASE
        WHEN (aafhtfun.FALTA / 60) > 9 THEN CAST((aafhtfun.FALTA / 60) AS STRING)
        ELSE '0' || CAST((aafhtfun.FALTA / 60) AS STRING)
     END) || ':' ||
     (CASE
        WHEN (aafhtfun.FALTA - ((aafhtfun.FALTA / 60) * 60)) > 9
            THEN CAST((aafhtfun.FALTA - ((aafhtfun.FALTA / 60) * 60)) AS STRING)
        ELSE '0' || CAST((aafhtfun.FALTA - ((aafhtfun.FALTA / 60) * 60)) AS STRING)
     END)                                                                   AS FALTA,

    /*------------------------------------------------------------------------*/
    aafhtfun.COMPENSADO                                                     AS MIN_COMPENSADO,
    (CASE
        WHEN (aafhtfun.COMPENSADO / 60) > 9 THEN CAST((aafhtfun.COMPENSADO / 60) AS STRING)
        ELSE '0' || CAST((aafhtfun.COMPENSADO / 60) AS STRING)
     END) || ':' ||
     (CASE
        WHEN (aafhtfun.COMPENSADO - ((aafhtfun.COMPENSADO / 60) * 60)) > 9
            THEN CAST((aafhtfun.COMPENSADO - ((aafhtfun.COMPENSADO / 60) * 60)) AS STRING)
        ELSE '0' || CAST((aafhtfun.COMPENSADO - ((aafhtfun.COMPENSADO / 60) * 60)) AS STRING)
     END)                                                                   AS COMPENSADO,

    /*------------------------------------------------------------------------*/
    aafhtfun.DESCANSO                                                       AS MIN_DESCANSO,
    (CASE
        WHEN (aafhtfun.DESCANSO / 60) > 9 THEN CAST((aafhtfun.DESCANSO / 60) AS STRING)
        ELSE '0' || CAST((aafhtfun.DESCANSO / 60) AS STRING)
     END) || ':' ||
     (CASE
        WHEN (aafhtfun.DESCANSO - ((aafhtfun.DESCANSO / 60) * 60)) > 9
            THEN CAST((aafhtfun.DESCANSO - ((aafhtfun.DESCANSO / 60) * 60)) AS STRING)
        ELSE '0' || CAST((aafhtfun.DESCANSO - ((aafhtfun.DESCANSO / 60) * 60)) AS STRING)
     END)                                                                   AS DESCANSO,

    /*------------------------------------------------------------------------*/
    aafhtfun.FERIADO                                                        AS MIN_FERIADO,
    (CASE
        WHEN (aafhtfun.FERIADO / 60) > 9 THEN CAST((aafhtfun.FERIADO / 60) AS STRING)
        ELSE '0' || CAST((aafhtfun.FERIADO / 60) AS STRING)
     END) || ':' ||
     (CASE
        WHEN (aafhtfun.FERIADO - ((aafhtfun.FERIADO / 60) * 60)) > 9
            THEN CAST((aafhtfun.FERIADO - ((aafhtfun.FERIADO / 60) * 60)) AS STRING)
        ELSE '0' || CAST((aafhtfun.FERIADO - ((aafhtfun.FERIADO / 60) * 60)) AS STRING)
     END)                                                                   AS FERIADO,

    /*------------------------------------------------------------------------*/
    aafhtfun.ADICIONAL                                                      AS MIN_ADICIONAL_NOTURNO,
    (CASE
        WHEN (aafhtfun.ADICIONAL / 60) > 9 THEN CAST((aafhtfun.ADICIONAL / 60) AS STRING)
        ELSE '0' || CAST((aafhtfun.ADICIONAL / 60) AS STRING)
     END) || ':' ||
     (CASE
        WHEN (aafhtfun.ADICIONAL - ((aafhtfun.ADICIONAL / 60) * 60)) > 9
            THEN CAST((aafhtfun.ADICIONAL - ((aafhtfun.ADICIONAL / 60) * 60)) AS STRING)
        ELSE '0' || CAST((aafhtfun.ADICIONAL - ((aafhtfun.ADICIONAL / 60) * 60)) AS STRING)
     END)                                                                   AS ADICIONAL_NOTURNO,

    /*------------------------------------------------------------------------*/
    CASE 
        WHEN EXISTS (
            SELECT 1
            FROM aabonfun a
            WHERE  a.CODCOLIGADA = aafhtfun.CODCOLIGADA
              AND  a.CHAPA       = aafhtfun.CHAPA
              AND  DATE(a.DATA)  = DATE(aafhtfun.DATA)
        ) THEN 'Sim'
        ELSE 'Não'
    END                                                                     AS STATUS_ABONO

FROM aafhtfun 
INNER JOIN pfunc  ON
    aafhtfun.CODCOLIGADA = pfunc.CODCOLIGADA
    AND aafhtfun.CHAPA   = pfunc.CHAPA
INNER JOIN tab_gold_dim_funcionario_historico AS fh ON
    pfunc.CODCOLIGADA = fh.CODCOLIGADA
    AND pfunc.CHAPA = fh.CHAPA
    AND aafhtfun.DATA = fh.DATA 
INNER JOIN psecao  ON
    pfunc.CODCOLIGADA    = psecao.CODCOLIGADA
    AND pfunc.CODSECAO   = psecao.CODIGO
INNER JOIN pfuncao ON 
    pfunc.CODCOLIGADA    = pfuncao.CODCOLIGADA
    AND pfunc.CODFUNCAO  = pfuncao.CODIGO    
INNER JOIN pfrateiofixo  ON
    pfrateiofixo.CHAPA       = pfunc.CHAPA
    AND pfrateiofixo.CODCOLIGADA = pfunc.CODCOLIGADA
INNER JOIN pccusto  ON
    pfrateiofixo.CODCCUSTO   = pccusto.CODCCUSTO
    AND pfrateiofixo.CODCOLIGADA = pccusto.CODCOLIGADA
WHERE
    aafhtfun.CODCOLIGADA IN ('1','2','3','4','5')
""")

# Salva no Lakehouse de destino usando caminho físico
df_resultado.write.format("delta").mode("overwrite").save(path_destino)


StatementMeta(, 485b5641-e11c-47da-993d-fa495996920f, 3, Finished, Available, Finished)